# TypeDB Jupyter
`typedb-jupyter` is a python library that introduces a few useful jupyter commands as well as python functions to enable users to work with TypeDB through jupyter notebooks, without having to pass through too much of the python driver.

The `%typedb` 'line magic' allows administrative server like user management, database management and transactional commands.
The `%%typeql` 'cell magic' runs a query within the active transaction.

To load the typedb-jupyter extension, we use:

In [1]:
%reload_ext typedb_jupyter

#### Open a connection

In [2]:
%typedb connect help

usage: connect [-h] [--tls-enabled]
               {open,close,help} [{core,cluster}] [address] [username]
               [password]

Establishes the connection to TypeDB

positional arguments:
  {open,close,help}
  {core,cluster}
  address
  username
  password

options:
  -h, --help         show this help message and exit
  --tls-enabled      Use for encrypted servers



In [3]:
%typedb connect open core 127.0.0.1:1729 admin password

Opened connection to: 127.0.0.1:1729


## Database Management


In [4]:
%typedb database help

usage: database [-h] {create,recreate,list,delete,schema,help} [name]

Database management

positional arguments:
  {create,recreate,list,delete,schema,help}
  name

options:
  -h, --help            show this help message and exit



In [5]:
%typedb database create typedb_jupyter_sample

Created database  typedb_jupyter_sample


In [6]:
%typedb database list

Databases:  tests, typedb_jupyter_sample, typedb_jupyter_graphs


In [7]:
%typedb database delete typedb_jupyter_sample

Deleted database  typedb_jupyter_sample


In [8]:
%typedb database list

Databases:  tests, typedb_jupyter_graphs


In [9]:
%typedb database recreate typedb_jupyter_sample

Recreated database  typedb_jupyter_sample


In [10]:
%typedb database list

Databases:  tests, typedb_jupyter_sample, typedb_jupyter_graphs


## Transactions & queries
To query TypeDB, one needs to use transactions. 
### Defining the schema
Open a `schema` transaction, define our schema, and commit.

In [11]:
%typedb transaction open typedb_jupyter_sample schema

Opened schema transaction on database 'typedb_jupyter_sample'  


In [12]:
%%typeql 
define
    attribute name, value string;
    entity person, owns name;


Query completed successfully! (No results to show)


'Stored result in variable: _typeql_result'

In [13]:
%typedb transaction commit

Transaction committed


### Writing data
Open a `write`, insert some data, and commit. 
Notice that the insert query does return the data.

In [14]:
%typedb transaction open typedb_jupyter_sample write

Opened write transaction on database 'typedb_jupyter_sample'  


In [15]:
%%typeql
insert 
$p isa person, has name "James";

Query returned 1 rows.


p
Entity(person: 0x1e00000000000000000000)


'Stored result in variable: _typeql_result'

In [16]:
%typedb transaction commit

Transaction committed


#### Reading data
We can read data through `match` queries, with a `fetch` at the end if desired. The collected result is stored automatically in the `_typeql_result` python variable

In [17]:
%typedb transaction open typedb_jupyter_sample read 

Opened read transaction on database 'typedb_jupyter_sample'  


In [18]:
%%typeql 
match $instance isa! $instance-type;

Query returned 2 rows.


instance,instance-type
Entity(person: 0x1e00000000000000000000),EntityType(person)
"Attribute(name: ""James"")",AttributeType(name)


'Stored result in variable: _typeql_result'

In [19]:
# Access the result through the _typeql_result variable
print(_typeql_result)
print(_typeql_result[1].get("instance"))

[|  $instance: Entity(person: 0x1e00000000000000000000)  |  $instance-type: EntityType(person)  |, |  $instance: Attribute(name: "James")  |  $instance-type: AttributeType(name)  |]
Attribute(name: "James")


In [20]:
%%typeql 
match $owner isa! $owner_type; entity $owner_type;
fetch {
    "attributes": { $owner.* }
};

Query returned 1 documents.
{
  "attributes": {
    "name": "James"
  }
}


'Stored result in variable: _typeql_result'

In [21]:
%typedb transaction close

Transaction closed


## Miscellaneous
One can list all available commands:

In [22]:
%typedb help

Available commands: connect, database, transaction, help
--------------------------------------------------------------------------------
Help for command 'connect':
usage: connect [-h] [--tls-enabled]
               {open,close,help} [{core,cluster}] [address] [username]
               [password]

Establishes the connection to TypeDB

positional arguments:
  {open,close,help}
  {core,cluster}
  address
  username
  password

options:
  -h, --help         show this help message and exit
  --tls-enabled      Use for encrypted servers

--------------------------------------------------------------------------------
Help for command 'database':
usage: database [-h] {create,recreate,list,delete,schema,help} [name]

Database management

positional arguments:
  {create,recreate,list,delete,schema,help}
  name

options:
  -h, --help            show this help message and exit

--------------------------------------------------------------------------------
Help for command 'transaction':
usage